In [ ]:
import pandas as pd
import numpy as np

programs = []

for i in range(1, 9):
    program = pd.read_csv('data/Program0%d.csv' % i)
    
    programs.append(program)
    
questions = pd.read_csv('data/Question.csv')

In [ ]:
import jieba 
jieba.set_dictionary('data/big5_dict.txt')

In [ ]:
def jieba_line(lines):
    cut_lines = []
    for line in lines:
        cut_line = jieba.lcut(line)
        cut_lines.append(cut_line)
        
    return cut_lines

In [ ]:
cut_programs = []

for program in programs:
    # number of lines in the program
    n = len(program)
    cut_program = []
    
    for i in range(n):
        lines = program.loc[i]['Content'].split('\n')
        cut_program.append(jieba_line(lines))
        
    cut_programs.append(cut_program)
    
print(len(cut_programs))
print(len(cut_programs[0]))
print(len(cut_programs[0][0]))
print(cut_programs[0][0][:3])



In [ ]:
cut_questions = []
n = len(questions)

for i in range(n):
    cut_question = []
    lines = questions.loc[i]['Question'].split('\n')
    cut_question.append(jieba_line(lines))
    
    for j in range(6):
        line = questions.loc[i]['Option%d' % (j)]
        cut_question.append(jieba.lcut(line))
    
    cut_questions.append(cut_question)
    
print(len(cut_questions))
print(len(cut_questions[0]))
print(len(cut_questions[0][0]))
print(cut_questions[0][0][:3])

In [ ]:
np.save('cut_Programs.npy', cut_programs)
np.save('cut_Questions.npy', cut_questions)